In [12]:
from langchain_core.prompts import (ChatPromptTemplate, 
                                    SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    AIMessagePromptTemplate,
                                    FewShotChatMessagePromptTemplate)

# Zero Shot Prompting

In [2]:
system_prompt = """Answer the question based on provided context. If you dont know the answer say I dont know. 
Context: {context} """


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}")
])

print(prompt.input_variables)
print([message.input_variables for message in prompt.messages])

['context', 'query']
[['context'], ['query']]


In [3]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the question based on provided context. If you dont know the answer say I dont know. \nContext: {context} '), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [4]:
prompt2 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt, input_variables=["context"]),
    HumanMessagePromptTemplate.from_template("{query}")
])

print([message.input_variables for message in prompt2.messages])

[['context'], ['query']]


In [6]:
context = "This is a sample text" 
query = "The response should be a random sequence of two words from the context only."

In [ ]:
chain = prompt | llm
chain.invoeke({"context": context, "query":query})

In [ ]:
pipeline = (
    {"context": lambda x: x["context"], "query": lambda x: x["query"]}
    | prompt
    | llm
)

# Few Shot Prompting

In [10]:
prompt3 = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

In [11]:
examples = [
    {"input": "This is query #1", "output": "This is output #1"},
    {"input": "This is query #2", "output": "This is output #2"}
]

In [13]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt3,
    examples=examples
)

In [14]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': 'This is query #1', 'output': 'This is output #1'}, {'input': 'This is query #2', 'output': 'This is output #2'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [16]:
print(few_shot_prompt.format())

Human: This is query #1
AI: This is output #1
Human: This is query #2
AI: This is output #2


In [17]:
final_prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    ("user", "{query}")
])

In [20]:
final_prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Answer the question based on provided context. If you dont know the answer say I dont know. \nContext: {context} '), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'This is query #1', 'output': 'This is output #1'}, {'input': 'This is query #2', 'output': 'This is output #2'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{ou